In [2]:
import numpy as np
import pandas as pd

In [3]:
train_YN = pd.read_csv("real_train_YN.csv")

def extract_target(df):
    target_id = train_YN['ad_id'].tolist()
    return df[df['ad_id'].isin(target_id)]

def extract_train(df, YN):
    train_id = train_YN[train_YN['train']==YN]['ad_id'].tolist()
    return df[df['ad_id'].isin(train_id)]

In [4]:
def drop_na_columns(df, percentage):
    missing_values_count = df.isnull().sum()

    # 데이터 세트의 값 중 몇 퍼센트가 누락되었는지 확인
    total_cells = np.product(df.shape)
    total_missing = missing_values_count.sum()

    print((total_missing/total_cells) * 100)

    # Percentage of NAN Values 
    NAN = [(c, df[c].isna().mean()*100) for c in df]
    NAN = pd.DataFrame(NAN, columns=["column_name", "percentage"])

    # Features with more than 50% of missing values.
    NAN = NAN[NAN.percentage > percentage]
    # NAN.sort_values("percentage", ascending=False)

    NAN_cols = NAN['column_name'].to_list()
    df_dropped_na = df.drop(NAN_cols,axis=1)
    print('Number of dropped columns', df.shape[1] - df_dropped_na.shape[1])
    
    return df_dropped_na


def drop_col_not_using(df):

    firebase_col = [s for s in df.columns if "firebase" in s]
    df_dropped_na = df.drop(firebase_col, axis=1)
    
    df_col = df_dropped_na.columns
    duplicated_col = [s for s in df_col if "_1" in s]
    app_info_col = [s for s in df_col if "app_info" in s]
    device_col = [s for s in df_col if "device" in s]
    traffic_col = [s for s in df_col if "traffic" in s]
    geo_col = [s for s in df_col if "geo" in s]
    # user_col =  [s for s in df_col if "user" in s]

    df_dropped_na.drop(duplicated_col, axis=1, inplace=True)
    df_dropped_na.drop(app_info_col, axis=1, inplace=True)
    df_dropped_na.drop(device_col, axis=1, inplace=True)
    df_dropped_na.drop(traffic_col, axis=1, inplace=True)
    df_dropped_na.drop(geo_col, axis=1, inplace=True)
    print('Number of dropped columns', df.shape[1] - df_dropped_na.shape[1])
    
    return df_dropped_na

## tutorial

In [ ]:
tutorial = pd.read_csv("data/tutorial_filtered.csv")

In [3]:
tutorial.shape

(993059, 50)

In [4]:
len(tutorial['ad_id'].unique())

240911

In [31]:
tutorial.shape

(993059, 50)

In [9]:
tutorial.columns

Index(['traffic_source_medium', 'traffic_source_source', 'traffic_source_name',
       'event_date', 'event_timestamp', 'event_name', 'ad_id', 'ga_session_id',
       'engaged_session_event', 'tutorial_name', 'ga_session_number',
       'tutorial_id', 'event_previous_timestamp', 'event_bundle_sequence_id',
       'event_server_timestamp_offset', 'user_pseudo_id',
       'ga_session_number_timestamp', 'ad_id_timestamp', 'cuid',
       'cuid_timestamp', 'frist_version', 'frist_version_timestamp',
       'user_level', 'user_level_timestamp', 'first_open_time',
       'first_open_time_timestamp', 'ga_session_id_timestamp',
       'user_first_touch_timestamp', 'device_category',
       'device_mobile_brand_name', 'mobile_model_name',
       'device_mobile_os_hardware_model', 'device_operating_system',
       'device_operating_system_version', 'device_advertising_id',
       'device_language', 'device_is_limited_ad_tracking',
       'device_time_zone_offset_seconds', 'geo_continent', 'geo_co

In [10]:
timestamp_col = ['event_previous_timestamp',
                 'event_server_timestamp_offset',
                 'ga_session_number_timestamp',
                 'ad_id_timestamp',
                 'cuid_timestamp',
                 'frist_version_timestamp',
                 'user_level_timestamp',
                 'first_open_time_timestamp',
                 'ga_session_id_timestamp',
                 'user_first_touch_timestamp']

In [32]:
tutorial.drop(timestamp_col, axis=1, inplace=True)

In [33]:
tutorial.drop(['geo_metro', 'items', 'device_is_limited_ad_tracking'], axis=1, inplace=True)

In [34]:
tutorial.to_csv('tutorial_filtered2.csv', index = False)

## ads

In [20]:
ads1 = pd.read_csv("data/brazil_ads_1to5.csv")
ads2 = pd.read_csv("data/brazil_ads_6to10.csv")

In [17]:
ads1.columns

Index(['items', 'traffic_source_medium', 'traffic_source_source',
       'traffic_source_name', 'event_previous_timestamp', 'ga_session_number',
       'ga_session_number_timestamp', 'user_level', 'user_level_timestamp',
       'frist_version', 'frist_version_timestamp', 'cuid', 'cuid_timestamp',
       'firebase_exp_8', 'firebase_exp_8_timestamp', 'firebase_exp_10',
       'firebase_exp_10_timestamp', 'ad_id', 'ad_id_timestamp',
       'firebase_exp_12', 'firebase_exp_12_timestamp', 'ga_session_id',
       'ga_session_id_timestamp', 'firebase_exp_4', 'firebase_exp_4_timestamp',
       'firebase_exp_6', 'firebase_exp_6_timestamp', 'first_open_time',
       'first_open_time_timestamp', 'firebase_exp_13',
       'firebase_exp_13_timestamp', 'firebase_exp_14',
       'firebase_exp_14_timestamp', 'firebase_exp_2',
       'firebase_exp_2_timestamp', 'firebase_exp_3',
       'firebase_exp_3_timestamp', 'firebase_last_notification',
       'firebase_last_notification_timestamp', 'last_gclid',

In [21]:
ads_col = ['event_previous_timestamp', 'event_timestamp', 'event_date',
           'ad_id', 'user_pseudo_id', 'ga_session_id', 'ga_session_number',
           'gubun', 'state', 'network', 'placement_id', 'placement_name', 
           'stage', 'Interval_time', 'user_level','cuid',
           'device_is_limited_ad_tracking']
ads1 = ads1[ads_col]
ads2 = ads2[ads_col]

In [22]:
ads = pd.concat([ads1, ads2], ignore_index=True)
ads.shape

(19000098, 17)

In [23]:
ads_datetime = pd.to_datetime(ads['event_timestamp'], unit='us')
ads['event_datetime'] = ads_datetime
ads_target = extract_target(ads)
len(ads_target['ad_id'].unique())

In [25]:
len(ads['ad_id'].unique())

243416

In [27]:
ads_target = extract_target(ads)
len(ads_target['ad_id'].unique())

In [30]:
ads_target.to_csv('ads_filtered.csv', index = False)

## activity

In [5]:
activity = pd.read_csv("data/brazil_activity.csv")

/home/users/s17011720/.conda/envs/jupyter/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (1,2,3,40) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
activity.columns

Index(['items', 'traffic_source_medium', 'traffic_source_source',
       'traffic_source_name', 'event_previous_timestamp', 'ga_session_number',
       'ga_session_number_timestamp', 'user_level', 'user_level_timestamp',
       'frist_version', 'frist_version_timestamp', 'cuid', 'cuid_timestamp',
       'firebase_exp_8', 'firebase_exp_8_timestamp', 'firebase_exp_10',
       'firebase_exp_10_timestamp', 'ad_id', 'ad_id_timestamp',
       'firebase_exp_12', 'firebase_exp_12_timestamp', 'ga_session_id',
       'ga_session_id_timestamp', 'firebase_exp_4', 'firebase_exp_4_timestamp',
       'firebase_exp_6', 'firebase_exp_6_timestamp', 'first_open_time',
       'first_open_time_timestamp', 'firebase_exp_13',
       'firebase_exp_13_timestamp', 'firebase_exp_14',
       'firebase_exp_14_timestamp', 'firebase_exp_2',
       'firebase_exp_2_timestamp', 'firebase_exp_3',
       'firebase_exp_3_timestamp', 'firebase_last_notification',
       'firebase_last_notification_timestamp', 'last_gclid',

In [7]:
activity_col = ['event_previous_timestamp', 'event_timestamp', 'event_date',
           'ad_id', 'user_pseudo_id', 'ga_session_id', 'ga_session_number',
           'activity_id', 'activity_name', 'action_id','action_name', 
           'sub_id', 'sub_name', 'user_level','cuid',
           'device_is_limited_ad_tracking']
activity = activity[activity_col]

In [8]:
act_datetime = pd.to_datetime(activity['event_timestamp'], unit='us')
activity['event_datetime'] = act_datetime

In [9]:
activity_target = extract_target(activity)
len(activity_target['ad_id'].unique())

144092

In [26]:
activity_target.to_csv('column_filtered_data/activity_filtered.csv', index = False)

## asset

In [11]:
asset = pd.read_csv("data/brazil_asset.csv")

/home/users/s17011720/.conda/envs/jupyter/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (1,2,3,19,43) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
asset.columns

Index(['items', 'traffic_source_medium', 'traffic_source_source',
       'traffic_source_name', 'event_previous_timestamp',
       'user_properties_ad_id', 'ad_id_timestamp', 'first_open_time',
       'first_open_time_timestamp', 'ga_session_id', 'ga_session_id_timestamp',
       'cuid', 'cuid_timestamp', 'user_level', 'user_level_timestamp',
       'frist_version', 'frist_version_timestamp', 'ga_session_number',
       'ga_session_number_timestamp', 'last_gclid', 'last_gclid_timestamp',
       'ads_abtest', 'ads_abtest_timestamp', 'ltv_USD', 'ltv_USD_timestamp',
       'ltv_BRL', 'ltv_BRL_timestamp', 'user_pseudo_id',
       'event_bundle_sequence_id', 'platform', 'event_server_timestamp_offset',
       'app_info_install_source', 'app_info_firebase_app_id',
       'app_info_version', 'app_info_id', 'event_date', 'event_name',
       'device_time_zone_offset_seconds', 'device_operating_system',
       'device_mobile_os_hardware_model', 'device_is_limited_ad_tracking',
       'device_ad

In [14]:
asset_col = ['event_previous_timestamp', 'event_timestamp', 'event_date',
           'user_properties_ad_id', 'event_params_ad_id', 'user_pseudo_id', 'ga_session_id', 'ga_session_number',
            'asset_id','asset_name', 'gubun', 'var', 'pre_var', 'curr_var',
             'action_id', 'action_name', 'item_id', 'item_name',
             'stage', 'user_level', 'cuid', 'device_is_limited_ad_tracking']
asset = asset[asset_col]

In [15]:
asset_datetime = pd.to_datetime(asset['event_timestamp'], unit='us')
asset['event_datetime'] = asset_datetime

/home/users/s17011720/.conda/envs/jupyter/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
asset.shape

(6953008, 23)

In [17]:
asset[['event_params_ad_id', 'user_properties_ad_id']].isnull().sum()

event_params_ad_id       63067
user_properties_ad_id    62762
dtype: int64

In [19]:
print('user_properties_ad_id : {}명'.format(len(asset['user_properties_ad_id'].unique())))
print('event_params_ad_id : {}명'.format(len(asset['event_params_ad_id'].unique())))

user_properties_ad_id : 241863명
event_params_ad_id : 241867명


In [20]:
params_ad_id_null = asset[asset['event_params_ad_id'].isnull()]
params_ad_id_null_idx = params_ad_id_null.index

In [21]:
params_ad_id_null['user_properties_ad_id'].unique()

array([nan, '11d108b1-4713-43b1-b6d8-dc5f7a7a451c',
       'de9526aa-ef77-4368-a5cb-01c3da5ef462',
       'd149a6b0-e7eb-4565-919a-0cf6dcc928ff',
       'da9b3634-2042-4b4d-b640-72123486d37f',
       'ca40815b-1341-4efb-b07c-407680081fdf',
       'a9038d84-f0a5-4717-89c1-85cb375dafd0',
       '6031d373-97d9-4a92-8a0b-1b949c7234f2',
       '150d4347-9f7b-4d7d-bd32-4d1aa9058900',
       'de005f3a-9c04-428d-85e0-df67d4e4f598',
       'db9db389-bc58-4370-a70f-b31fefc6372d',
       'ce2f4c2c-90f6-449a-8741-de13c2b3b743',
       'd06b8321-895d-4670-9f56-e2aa4908d3e1',
       '900cd2b1-253b-408c-ab6c-cb26234cdc38',
       '0a2d0f69-1fc3-4536-94d2-55d436980837',
       '60f0be35-8415-4b4b-82ff-48cb9c147234',
       '6ac5ccbf-8bfc-4c9e-b065-e933f303a317',
       '41004ed5-0299-4391-9705-bea267bf0e0a',
       '06418090-2770-422a-888c-93ef6333bce5',
       'f3ea4820-8d18-4d66-b43c-c995f3de0dd0',
       '4207ceb2-7590-468c-9337-d021e38e7683',
       'fbe10816-39f1-401c-ad4e-7a910f73d305',
       '

In [22]:
asset.loc[params_ad_id_null_idx, 'event_params_ad_id'] = asset.loc[params_ad_id_null_idx, 'user_properties_ad_id']

In [23]:
asset.drop('user_properties_ad_id', axis=1, inplace=True)
asset.rename(columns={'event_params_ad_id': 'ad_id'}, inplace=True)

In [24]:
asset_target = extract_target(asset)
len(asset_target['ad_id'].unique())

224885

In [25]:
asset_target.to_csv('asset_filtered.csv', index = False)